In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import time

from datetime import datetime as dt
from bs4 import BeautifulSoup

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
USERNAME = "postgres"
PASSWORD = "georgia1"
rds_connection_string = f"{USERNAME}:{PASSWORD}@localhost:5432/EColi"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
file2015 = "2015.csv"
file2016 = "2016.csv"
file2017 = "2017.csv"
file2018 = "2018.csv"
file2019 = "2019.csv"
df15 = pd.read_csv(file2015)
df16 = pd.read_csv(file2016)
df17 = pd.read_csv(file2017)
df18 = pd.read_csv(file2018)
df19 = pd.read_csv(file2019)

In [4]:
df15 = df15.drop(columns=['Region', 'Happiness Rank', 'Standard Error', 'Family', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual'])

In [5]:
df15.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy)
0,Switzerland,7.587,1.39651,0.94143
1,Iceland,7.561,1.30232,0.94784
2,Denmark,7.527,1.32548,0.87464
3,Norway,7.522,1.45900,0.88521
4,Canada,7.427,1.32629,0.90563


In [6]:
df15['Year'] = '2015'
df15.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Year
0,Switzerland,7.587,1.39651,0.94143,2015
1,Iceland,7.561,1.30232,0.94784,2015
2,Denmark,7.527,1.32548,0.87464,2015
3,Norway,7.522,1.45900,0.88521,2015
4,Canada,7.427,1.32629,0.90563,2015


In [7]:
df16 = df16.drop(columns=['Region', 'Happiness Rank', 'Lower Confidence Interval', 'Upper Confidence Interval', 'Family', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual'])
df16.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy)
0,Denmark,7.526,1.44178,0.79504
1,Switzerland,7.509,1.52733,0.86303
2,Iceland,7.501,1.42666,0.86733
3,Norway,7.498,1.57744,0.79579
4,Finland,7.413,1.40598,0.81091


In [8]:
df16['Year'] = '2016'
df16.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Year
0,Denmark,7.526,1.44178,0.79504,2016
1,Switzerland,7.509,1.52733,0.86303,2016
2,Iceland,7.501,1.42666,0.86733,2016
3,Norway,7.498,1.57744,0.79579,2016
4,Finland,7.413,1.40598,0.81091,2016


In [9]:
df17 = df17.drop(columns=['Happiness.Rank', 'Whisker.high', 'Whisker.low', 'Family', 'Freedom', 'Generosity', 'Trust..Government.Corruption.', 'Dystopia.Residual'])
df17.head()

,Country,Happiness.Score,Economy..GDP.per.Capita.,Health..Life.Expectancy.
0,Norway,7.537,1.616463,0.796667
1,Denmark,7.522,1.482383,0.792566
2,Iceland,7.504,1.480633,0.833552
3,Switzerland,7.494,1.564980,0.858131
4,Finland,7.469,1.443572,0.809158


In [10]:
df17 = df17.rename({'Happiness.Score': 'Happiness Score', 'Economy..GDP.per.Capita.': 'Economoy (GDP per Capita)', 'Health..Life.Expectancy.': 'Health (Life Expectancy)'}, axis=1)
df17.head()

,Country,Happiness Score,Economoy (GDP per Capita),Health (Life Expectancy)
0,Norway,7.537,1.616463,0.796667
1,Denmark,7.522,1.482383,0.792566
2,Iceland,7.504,1.480633,0.833552
3,Switzerland,7.494,1.564980,0.858131
4,Finland,7.469,1.443572,0.809158


In [11]:
df17['Year'] = '2017'
df17.head()

,Country,Happiness Score,Economoy (GDP per Capita),Health (Life Expectancy),Year
0,Norway,7.537,1.616463,0.796667,2017
1,Denmark,7.522,1.482383,0.792566,2017
2,Iceland,7.504,1.480633,0.833552,2017
3,Switzerland,7.494,1.564980,0.858131,2017
4,Finland,7.469,1.443572,0.809158,2017


In [13]:
df18 = df18.drop(columns=['Overall rank', 'Social support', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption'])
df18.head()

,Country or region,Score,GDP per capita,Healthy life expectancy
0,Finland,7.632,1.305,0.874
1,Norway,7.594,1.456,0.861
2,Denmark,7.555,1.351,0.868
3,Iceland,7.495,1.343,0.914
4,Switzerland,7.487,1.420,0.927


In [15]:
df18 = df18.rename({'Country or region': 'Country', 'Score': 'Happiness Score', 'GDP per capita': 'Economy (GDP per Capita)', 'Healthy life expectancy': 'Health (Life Expectancy)'}, axis=1)
df18.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy)
0,Finland,7.632,1.305,0.874
1,Norway,7.594,1.456,0.861
2,Denmark,7.555,1.351,0.868
3,Iceland,7.495,1.343,0.914
4,Switzerland,7.487,1.420,0.927


In [16]:
df18['Year'] = '2018'
df18.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Year
0,Finland,7.632,1.305,0.874,2018
1,Norway,7.594,1.456,0.861,2018
2,Denmark,7.555,1.351,0.868,2018
3,Iceland,7.495,1.343,0.914,2018
4,Switzerland,7.487,1.420,0.927,2018


In [17]:
df19 = df19.drop(columns=['Overall rank', 'Social support', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption'])
df19.head()

,Country or region,Score,GDP per capita,Healthy life expectancy
0,Finland,7.769,1.340,0.986
1,Denmark,7.600,1.383,0.996
2,Norway,7.554,1.488,1.028
3,Iceland,7.494,1.380,1.026
4,Netherlands,7.488,1.396,0.999


In [18]:
df19 = df19.rename({'Country or region': 'Country', 'Score': 'Happiness Score', 'GDP per capita': 'Economy (GDP per Capita)', 'Healthy life expectancy': 'Health (Life Expectancy)'}, axis=1)
df19.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy)
0,Finland,7.769,1.340,0.986
1,Denmark,7.600,1.383,0.996
2,Norway,7.554,1.488,1.028
3,Iceland,7.494,1.380,1.026
4,Netherlands,7.488,1.396,0.999


In [19]:
df19['Year'] = '2019'
df19.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Year
0,Finland,7.769,1.340,0.986,2019
1,Denmark,7.600,1.383,0.996,2019
2,Norway,7.554,1.488,1.028,2019
3,Iceland,7.494,1.380,1.026,2019
4,Netherlands,7.488,1.396,0.999,2019


In [32]:
happiness_df = df15
happiness_df.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Year
0,Switzerland,7.587,1.39651,0.94143,2015
1,Iceland,7.561,1.30232,0.94784,2015
2,Denmark,7.527,1.32548,0.87464,2015
3,Norway,7.522,1.45900,0.88521,2015
4,Canada,7.427,1.32629,0.90563,2015


In [34]:
happiness_df = happiness_df.append(df16)
happiness_df

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Year
0,Switzerland,7.587,1.39651,0.94143,2015
1,Iceland,7.561,1.30232,0.94784,2015
2,Denmark,7.527,1.32548,0.87464,2015
3,Norway,7.522,1.45900,0.88521,2015
4,Canada,7.427,1.32629,0.90563,2015
...,...,...,...,...,...
152,Benin,3.484,0.39499,0.21028,2016
153,Afghanistan,3.360,0.38227,0.17344,2016
154,Togo,3.303,0.28123,0.24811,2016
155,Syria,3.069,0.74719,0.62994,2016


In [35]:
happiness_df = happiness_df.append(df17)
happiness_df

,Country,Economoy (GDP per Capita),Economy (GDP per Capita),Happiness Score,Health (Life Expectancy),Year
0,Switzerland,NaN,1.39651,7.587,0.941430,2015
1,Iceland,NaN,1.30232,7.561,0.947840,2015
2,Denmark,NaN,1.32548,7.527,0.874640,2015
3,Norway,NaN,1.45900,7.522,0.885210,2015
4,Canada,NaN,1.32629,7.427,0.905630,2015
...,...,...,...,...,...,...
150,Rwanda,0.368746,NaN,3.471,0.326425,2017
151,Syria,0.777153,NaN,3.462,0.500533,2017
152,Tanzania,0.511136,NaN,3.349,0.364509,2017
153,Burundi,0.091623,NaN,2.905,0.151611,2017


In [36]:
happiness_df = happiness_df.append(df18)
happiness_df

,Country,Economoy (GDP per Capita),Economy (GDP per Capita),Happiness Score,Health (Life Expectancy),Year
0,Switzerland,NaN,1.39651,7.587,0.94143,2015
1,Iceland,NaN,1.30232,7.561,0.94784,2015
2,Denmark,NaN,1.32548,7.527,0.87464,2015
3,Norway,NaN,1.45900,7.522,0.88521,2015
4,Canada,NaN,1.32629,7.427,0.90563,2015
...,...,...,...,...,...,...
151,Yemen,NaN,0.44200,3.355,0.34300,2018
152,Tanzania,NaN,0.45500,3.303,0.38100,2018
153,South Sudan,NaN,0.33700,3.254,0.17700,2018
154,Central African Republic,NaN,0.02400,3.083,0.01000,2018


In [37]:
happiness_df = happiness_df.append(df19)
happiness_df

,Country,Economoy (GDP per Capita),Economy (GDP per Capita),Happiness Score,Health (Life Expectancy),Year
0,Switzerland,NaN,1.39651,7.587,0.94143,2015
1,Iceland,NaN,1.30232,7.561,0.94784,2015
2,Denmark,NaN,1.32548,7.527,0.87464,2015
3,Norway,NaN,1.45900,7.522,0.88521,2015
4,Canada,NaN,1.32629,7.427,0.90563,2015
...,...,...,...,...,...,...
151,Rwanda,NaN,0.35900,3.334,0.61400,2019
152,Tanzania,NaN,0.47600,3.231,0.49900,2019
153,Afghanistan,NaN,0.35000,3.203,0.36100,2019
154,Central African Republic,NaN,0.02600,3.083,0.10500,2019


In [38]:
happiness_df = happiness_df.sort_values('Country')
happiness_df

,Country,Economoy (GDP per Capita),Economy (GDP per Capita),Happiness Score,Health (Life Expectancy),Year
153,Afghanistan,NaN,0.38227,3.360,0.173440,2016
152,Afghanistan,NaN,0.31982,3.575,0.303350,2015
144,Afghanistan,NaN,0.33200,3.632,0.255000,2018
153,Afghanistan,NaN,0.35000,3.203,0.361000,2019
140,Afghanistan,0.401477,NaN,3.794,0.180747,2017
...,...,...,...,...,...,...
143,Zimbabwe,NaN,0.35700,3.692,0.248000,2018
145,Zimbabwe,NaN,0.36600,3.663,0.433000,2019
114,Zimbabwe,NaN,0.27100,4.610,0.334750,2015
130,Zimbabwe,NaN,0.35041,4.193,0.159500,2016


In [41]:
happiness_df.to_csv('happiness_df.csv') 

In [42]:
alcohol_cons = "HappinessAlcoholConsumption.csv"
alcohol_df = pd.read_csv(alcohol_cons)
alcohol_df.head()

,Country,Region,Hemisphere,HappinessScore,HDI,GDP_PerCapita,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
0,Denmark,Western Europe,north,7.526,928,53.579,224,81,278
1,Switzerland,Western Europe,north,7.509,943,79.866,185,100,280
2,Iceland,Western Europe,north,7.501,933,60.530,233,61,78
3,Norway,Western Europe,north,7.498,951,70.890,169,71,129
4,Finland,Western Europe,north,7.413,918,43.433,263,133,97


In [43]:
alcohol_df.drop(columns=['Region', 'Hemisphere', 'HappinessScore', 'HDI', 'GDP_PerCapita'])

,Country,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
0,Denmark,224,81,278
1,Switzerland,185,100,280
2,Iceland,233,61,78
3,Norway,169,71,129
4,Finland,263,133,97
...,...,...,...,...
117,Tanzania,36,6,1
118,Liberia,19,152,2
119,Benin,34,4,13
120,Togo,36,2,19


In [44]:
alcohol_df.to_csv('alcohol_df.csv') 